<a href="https://colab.research.google.com/github/Aniket-Mitra/Meta-N-Ball/blob/main/Preprocessing_%26_Data_Preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Extracting Metadata-Info (Synonyms)

In [ ]:
list_line2=[]
with open("hp.obo") as f:  #we will extract metadata info from .obo file
        for k,line in enumerate(f):
          x = line[:-1]
          list_line2.append(x)
#list_line2[-1]=list_line2[-1]+')'
#list_line2[-1]
list_line2[-2]

In [ ]:
def create_metaaxioms(type1='EXACT'):
  if type1=='EXACT [':
    url='http://www.geneontology.org/formats/oboInOwl#hasExactSynonym'
  elif type1=='NARROW [':
    url='http://www.geneontology.org/formats/oboInOwl#hasNarrowSynonym'
  elif type1=='BROAD [':
    url='http://www.geneontology.org/formats/oboInOwl#hasBroadSynonym'
  elif type1=='RELATED [':
    url='http://www.geneontology.org/formats/oboInOwl#hasRelatedSynonym'
  dict_all=[]
  count=-1
  for i in list_line2:
   count+=1
   if type1 in i:
    exact_syn=i.split('"')
    for ij in range(count,0,-1):
          if list_line2[ij].startswith('id:'):
            id=list_line2[ij].split(' ')[1]
            id=id.replace(':','_')
            break
    axioms1='http://purl.obolibrary.org/obo/'+id+' '+url+' '+exact_syn[1]
    dict_all.append(axioms1)
  return dict_all

In [ ]:
dict_all_exact=create_metaaxioms(type1='EXACT [')
dict_all_exact=list(set(dict_all_exact))
print(dict_all_exact[-1])
print(len(dict_all_exact))

In [ ]:
dict_all_narrow=create_metaaxioms(type1='NARROW [')
dict_all_narrow=list(set(dict_all_narrow))
print(dict_all_narrow[-1])
print(len(dict_all_narrow))

In [ ]:
dict_all_broad=create_metaaxioms(type1='BROAD [') # run this for all the 4 types of synonyms- BROAD, EXACT, RELATED, NARROW
dict_all_broad=list(set(dict_all_broad))
print(dict_all_broad[-1])
print(len(dict_all_broad))

In [ ]:
dict_annots=dict_all_exact+dict_all_narrow+dict_all_broad+dict_all_related # append the list of all the types of synonyms obtained

In [ ]:
ts='\n'.join(dict_annots)
file = open('annotations_code2_hp.txt', 'w') # the extracted synonym information is stored in this file
file.write(ts)
file.close()

# Metadata-Term Extraction NLP Model

In [ ]:
mod3=[]
with open(r'annotations_code2_hp.txt', 'r') as fp:
    for line in fp:
        # remove linebreak from a current name
        # linebreak is the last character of each line
        #if '0002481' in line :
           x = line[:-1]

        # add current item to the list
           mod3.append(x)

In [ ]:
def get_axioms(type1,index_length):
   if type1=='hasExactSynonym':
    url='http://www.geneontology.org/formats/oboInOwl#hasExactSynonym'
   elif type1=='hasNarrowSynonym':
    url='http://www.geneontology.org/formats/oboInOwl#hasNarrowSynonym'
   elif type1=='hasBroadSynonym':
    url='http://www.geneontology.org/formats/oboInOwl#hasBroadSynonym'
   elif type1=='hasRelatedSynonym':
    url='http://www.geneontology.org/formats/oboInOwl#hasRelatedSynonym'

   narrow_axioms=[]
   narrow_list={}
   for line in mod3:
      if type1 in line and 'rdf-schema#label' not in line:
    #print(i)
          def_index=line.index(url)
          class_name=line[:def_index-1]
          defs=line[def_index*2+index_length:]
    #defs=line[def_index*2+21:]
          defs=defs.replace(' ','_')
    #narrow_list.append(class_name+defs)
          if defs not in narrow_list:
               narrow_list[defs]=[class_name]
          else:
               narrow_list[defs].append(class_name)
   return narrow_list

#narrow_list=get_axioms(type1='hasNarrowSynonym',index_length=20)
#narrow_list=get_axioms(type1='hasBroadSynonym',index_length=19)
#narrow_list=get_axioms(type1='hasRelatedSynonym',index_length=21)
narrow_list=get_axioms(type1='hasExactSynonym',index_length=19)

In [ ]:
!pip install scispacy

!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.3/en_core_sci_md-0.5.3.tar.gz

In [ ]:
for i in narrow_list:
  if len(narrow_list[i])>1:
     print(narrow_list[i],i)
     narrow_list[i]=[' '.join(narrow_list[i]) ]

import pandas as pd
df3=pd.DataFrame(narrow_list).T.reset_index()
df3.rename(columns={"index":'metadata',0:'url'},inplace=True)
df3.head()

In [ ]:
#df2=pd.DataFrame()
df2=pd.concat([df2,df3])
df2.head()

In [ ]:
df2['metadata']=df2.metadata.str.replace('_',' ').str.lower()
df2.head()

In [ ]:
dfab2temp=df2.copy()

In [ ]:
import scispacy
import spacy
#dfab2temp.drop(['cleaned_str_scispacymd'],axis=1,inplace=True)
nlp = spacy.load("en_core_sci_md")

list_tokens_labels={}
list_tokens_labels2=[]
#nlp = spacy.load("en_core_sci_md")
for i in range(len(dfab2temp)):
  hv_temp_list=[]
  text=dfab2temp.iloc[i,0]
  doc = nlp(text)
  #list_tokens_labels[dfab2temp.iloc[i,-1]]=doc.ents
  list_tokens_labels2.append(list(set(doc.ents)))

dfab2temp['cleaned_str_scispacymd']= list_tokens_labels2

/usr/local/lib/python3.10/dist-packages/spacy/language.py:2141: FutureWarning: Possible set union at position 6328
  deserializers["tokenizer"] = lambda p: self.tokenizer.from_disk(  # type: ignore[union-attr]


In [ ]:
list2=dfab2temp['cleaned_str_scispacymd'].tolist()

list3=[]
for i in list2:
  list_t=[]
  for j in i:
      list_t.append(str(j))
  list3.append(list_t)

list3a=[]
for i in list3:
  list_t=''
  for j in i:
      list_t=list_t+j+' '
  list3a.append(list_t)

In [ ]:
dfab2temp['cleaned_str_scispacymd2']=list3a

In [ ]:
a1=dfab2temp.groupby('cleaned_str_scispacymd2')['url'].count().reset_index(name='count').sort_values(['count'], ascending=False)['cleaned_str_scispacymd2'].tolist()
a1=[i for i in a1 if len(i)>0]

In [ ]:
dfab2temp2=dfab2temp[dfab2temp['cleaned_str_scispacymd2'].isin(a1)]

In [ ]:
dfab2temp4=dfab2temp2.copy()
dfab2temp4['urls']=dfab2temp4['url'].str.split(' ')
dfab2temp5=dfab2temp4.explode('url')
dfab2temp5=dfab2temp5[['url','cleaned_str_scispacymd2']]
#dfab2temp5.drop_duplicates(inplace=True)
dfab2temp5.head()

In [ ]:
dfab2temp5.to_csv('final_all_syns5_hpo.csv',index=None) # extracted metadata by nlp model for each class stored in csv format

# Adding Label Information and Creating Finalized Normal OWL File

In [ ]:
import pandas as pd

dfab2temp5=pd.read_csv('final_all_syns5_hpo.csv')

dfab2temp_c=dfab2temp5.copy()

dfab2temp5=dfab2temp5[['url','cleaned_str_scispacymd2']]

In [ ]:
list_alln=[]

for i in range(len(dfab2temp5)):
      #ab=[dfab2temp5.iloc[i,1][1:-1].split(',')]
      ab=[dfab2temp5.iloc[i,1].split(',')]
      list_alln.extend(ab)

dfab2temp5['cleaned_str_scispacymd2']=list_alln

dfab2url2=dfab2temp5.explode('cleaned_str_scispacymd2')
dfab2url2.head(3)

In [ ]:
dfab2url2['cleaned_str_scispacymd2']=dfab2url2['cleaned_str_scispacymd2'].str.replace("'",'').str.lstrip()

In [ ]:
dfab2url2=dfab2url2[dfab2url2['cleaned_str_scispacymd2']!='']
dfab2url2.head()

In [ ]:
#dfab2url2.to_csv('extracted_hpo_ins.csv',index=None)

In [ ]:
mod3=[]
with open(r'hp_8s11feb24_norm2.owl', 'r') as fp:  #normalized owl file by normalizer.groovy
    for line in fp:
        # remove linebreak from a current name
        # linebreak is the last character of each line
        #if '0002481' in line :
        if 'SubObjectPropertyOf' not in line:
           x = line[:-1]

        # add current item to the list
           mod3.append(x)

unique_tokens=list(dfab2url2['cleaned_str_scispacymd2'].unique())

dict_hv2={}
for i in unique_tokens:
      dict_hv2[i]={'nf1':0,'nf2':0,'nf3':0,'nf4':0,'nf5':0}

dict_del_nf={}
for i in unique_tokens:
      dict_del_nf[i]=[]

dict_nf1={}
for i in unique_tokens:
      dict_nf1[i]=[]

for i in unique_tokens:
    temp_df=dfab2url2[dfab2url2['cleaned_str_scispacymd2']==i]
    temp_df_l=temp_df['url'].tolist()
    #print(temp_df_l[0])
    for j in temp_df_l:
      for k in mod3:
            if j in k and 'Object' not in k:
                     dict_hv2[i]['nf1']+=1
                     kj=k[11:-1].split()
                     dict_nf1[i].append([kj[0][1:-1],kj[1][1:-1]])

            if j in k and 'ObjectIntersectionOf' in k:
                     dict_hv2[i]['nf2']+=1
                     kall=k[32:].replace(')','').replace('<','').replace('>','').split()
                     for kaa in kall:
                      if kaa not in dict_del_nf[i]:
                        dict_del_nf[i].append(kaa)

            if j in k and 'owl:Nothing' in k:
                     dict_hv2[i]['nf5']+=1
                     #dict_del_nf[i]
            if j in k and k[11]!='O' and 'ObjectSomeValuesFrom' in k:
                     dict_hv2[i]['nf3']+=1
                     if k.split('<')[1][:41] not in dict_del_nf[i]:
                         dict_del_nf[i].append(k.split('<')[1][:41])
                     if k.split('<')[3][:-3] not in dict_del_nf[i]:
                         dict_del_nf[i].append(k.split('<')[3][:-3])
            if j in k and k[11]=='O' and 'ObjectSomeValuesFrom' in k:
                     dict_hv2[i]['nf4']+=1
                     if k[32:-1].split(' ')[1][1:-2] not in dict_del_nf[i]:
                         dict_del_nf[i].append(k[32:-1].split(' ')[1][1:-2])
                     if k[32:-1].split(' ')[2][1:-1] not in dict_del_nf[i]:
                         dict_del_nf[i].append(k[32:-1].split(' ')[2][1:-1])

In [ ]:
list_allb=[]
for i in dict_nf1:
    for j in dict_nf1[i]:
      list_allb.append(j)
      #print(j)
      #list_allbc.append(j[0])
      #list_allbd.append(j[1])

list_allbc=[i[0] for i in list_allb]
list_allbd=[i[1] for i in list_allb]

all_key_nf1=[]
for i in dict_nf1:
       a1=[i]*len(list(dict_nf1[i]))
       for j in a1:
             all_key_nf1.append(j)

df_nf1_2=pd.DataFrame(all_key_nf1,columns=['cleaned_str_scispacymd2'])
df_nf1_2['subclass_c']=list_allbc
df_nf1_2['subclass_d']=list_allbd

In [ ]:
list_af=[]
for i in list(dict_del_nf.values()):
  #print(i)
  #print('---------')
  list_af.append(i)

pd.DataFrame(list_af).T

all_key=[]
for i in dict_del_nf:
       a1=[i]*len(list(dict_del_nf[i]))
       for j in a1:
             all_key.append(j)


all_vals=[]
for i in dict_del_nf:
  list_val=list(dict_del_nf[i])
  for j in list_val:
    all_vals.append(j)

print(len(all_key),len(all_vals))


df_nf=pd.DataFrame(all_key)
df_nf.columns=['cleaned_str_scispacymd2']
df_nf['urls']=all_vals

df_nf.head()

In [ ]:
##optional

dfab2url2.rename(columns={'url':'urls'},inplace=True)

In [ ]:
df_commons=pd.merge(left=dfab2url2,right=df_nf,on=['cleaned_str_scispacymd2','urls'],how='inner').drop_duplicates()
#df_commons.head()

df_fnf1=pd.merge(left=df_commons,right=df_nf1_2,how='inner',on=['cleaned_str_scispacymd2']).drop_duplicates()
df_nf1f=df_fnf1.drop('urls',axis=1).drop_duplicates()



In [ ]:
dfab2temp4=dfab2url2.copy()
dfab2temp4['url']=dfab2temp4['url'].str.split(' ')
dfab2temp5=dfab2temp4.explode('url')


In [ ]:
save_axioms=[]
#for i in range(len(df_nf1f2)):
for i in range(len(dfab2temp5)):
  #aa='InstanceOf(<'+df_nf1f2.iloc[i,0]+'> <'+df_nf1f2.iloc[i,1]+'> <'+df_nf1f2.iloc[i,2]+'>)'
  aa='InstanceOf(<'+dfab2temp5.iloc[i,1]+'> <'+dfab2temp5.iloc[i,0]+'> <'+dfab2temp5.iloc[i,0]+'>)'
  save_axioms.append(aa)
#save_axioms[-1]


go_or=[]
with open(r'hp_8s11feb24_norm2.owl', 'r') as fp:
    for line in fp:
        # remove linebreak from a current name
        # linebreak is the last character of each line
        #if '0002481' in line :
           x = line[:-1]

        # add current item to the list
           go_or.append(x)
#go_or[-1]

#all_axioms_f=go_or+instance_axiom
all_axioms_f=go_or+save_axioms
all_axiom=list(set(all_axioms_f))
print(len(all_axiom))


ts='\n'.join(all_axiom)
file = open('HPO_metadata_finalnf1_26Feb_all.owl', 'w') #Final owl file for training
file.write(ts)  #141134
file.close()

# Test-Train-Split

In [ ]:
SAVE_PATH="{}.{}"
FILE_NAME="HPO_metadata_finalnf1_26Feb_all.owl"  #name of finalized ontology (.owl)



In [ ]:


file_path = FILE_NAME
  #print(SPLIT)
file = {}
  # count = {}
for k in ['train', 'valid', 'test']:

    #file[k] = open(SAVE_PATH.format(DATASET, SPLIT, k, "txt"), 'w')
    file[k] = open(SAVE_PATH.format( k, "txt"), 'w')
    # count[k] = 0
file['train_norm'] = open(SAVE_PATH.format( "train_norm", "owl"), 'w')
  # count['train_norm'] = 0
subclass_axioms = []
other_axioms = []
co=0
train = 0
valid = 0
test = 0
classes = {}
relations = {}
subclass_classes = {}
  # First pass: Get all classes in subclass axioms, number of axioms
with open(file_path) as f:
    for line in f:
      co+=1
      og_line = line
      if line.startswith("SubClassOf"):
        line = line.strip()[11:-1]
        if not line:
          co-=1
          continue
        if not line.startswith("ObjectIntersectionOf(") and not line.startswith("ObjectSomeValuesFrom(") and line.find("ObjectSomeValuesFrom(") == -1:
          # SubClassOf C D
          it = line.split(' ')
          c = it[0]
          d = it[1]
          # subclass_axioms.append((og_line, c, d))
          if c not in subclass_classes:
            subclass_classes[c] = len(subclass_classes)
          if d not in subclass_classes:
            subclass_classes[d] = len(subclass_classes)

train = int(co*0.7)
valid = int(co*0.3)
test = co - train- valid
if test>0:
     valid+=test
     test=0
ex_train = train
total_train = train
count=0
print("total", co)
print("Expected train, valid, test:", train, valid, test)

  # Second pass: add all relation (rbox) axioms and only add non subclass axioms containing classes in subclass axioms
with open(file_path) as f:
    for line in f:
      og_line = line
      if line.startswith('SubObjectPropertyOf'):
        line = line.strip()[20:-1]
        if line.startswith('ObjectPropertyChain'):
          line_chain = line.strip()[20:-1]
          line1 = line.split(")")
          line10 = line1[0].split()
          if len(line10) < 2:
            continue
          r1 = line10[0]
          r2 = line10[1]
          r3 = line1[1]
          # if train and (r1 not in relations or r2 not in relations or r3 not in relations):
          file['train_norm'].write(og_line)
          file['train'].write(r1 + ' ' + r2 + '\n')
          count+=1
          train-=1
            # if r1 not in relations:
            #   relations[r1] = len(relations)
            # if r2 not in relations:
            #   relations[r2] = len(relations)
            # if r3 not in relations:
            #   relations[r3] = len(relations)
          # else:
            # other_axioms.append((og_line, r1, r2, r3))
        else:
          # print("Inside sub obj prop")
          it = line.split(' ')
          r1 = it[0]
          r2 = it[1]
          file['train_norm'].write(og_line)
          file['train'].write(r1 + ' ' + r2 + '\n')
          count+=1
          train-=1
          # other_axioms.append((og_line, r1, r2, ""))
        continue
      line = line.strip()[11:-1]
#           print(line)
      if not line:
        print(og_line)
        continue
      if line.startswith('ObjectIntersectionOf('):
        # C and D SubClassOf E
        it = line.split(' ')
        c = it[0][21:]
        d = it[1][:-1]
        e = it[2]
        if train and (c in subclass_classes or d in subclass_classes or e in subclass_classes):
          file['train_norm'].write(og_line)
          file['train'].write(c + ' ' + d + '\n')
          count+=1
          train-=1
          if c in subclass_classes:
            subclass_classes.pop(c)
          if d in subclass_classes:
            subclass_classes.pop(d)
          if e in subclass_classes:
            subclass_classes.pop(e)
          if c not in classes:
            classes[c] = len(classes)
          if d not in classes:
            classes[d] = len(classes)
          if e not in classes:
            classes[e] = len(classes)
        else:
          other_axioms.append((og_line, c, d, e))
      elif line.startswith('ObjectSomeValuesFrom('):
        # R some C SubClassOf D
        it = line.split(' ')
        r = it[0][21:]
        c = it[1][:-1]
        d = it[2]
        if train and (c in subclass_classes or d in subclass_classes or r not in relations):
          file['train_norm'].write(og_line)
          file['train'].write(c + ' ' + d + '\n')
          count+=1
          train-=1
          if c in subclass_classes:
            subclass_classes.pop(c)
          if d in subclass_classes:
            subclass_classes.pop(d)
          if c not in classes:
            classes[c] = len(classes)
          if d not in classes:
            classes[d] = len(classes)
          if r not in relations:
            relations[r] = len(relations)
        else:
          other_axioms.append((og_line, r, c, d))
      elif line.find('ObjectSomeValuesFrom') != -1:
        # C SubClassOf R some D
        it = line.split(' ')
        c = it[0]
        r = it[1][21:]
        d = it[2][:-1]
        if train and (c in subclass_classes or d in subclass_classes or r not in relations):
          file['train_norm'].write(og_line)
          file['train'].write(c + ' ' + d + '\n')
          count+=1
          train-=1
          if c in subclass_classes:
            subclass_classes.pop(c)
          if d in subclass_classes:
            subclass_classes.pop(d)
          if c not in classes:
            classes[c] = len(classes)
          if d not in classes:
            classes[d] = len(classes)
          if r not in relations:
            relations[r] = len(relations)
        else:
          other_axioms.append((og_line, r, c, d))
      else:
        # C SubClassOf D
        it = line.split(' ')
        c = it[0]
        d = it[1]
        subclass_axioms.append((og_line, c, d))
print("Added to train from other axioms", ex_train-train, count)
print(count+len(subclass_axioms)+len(other_axioms), len(subclass_axioms), len(other_axioms))
print("Classes not added", len([k for k in subclass_classes if k not in classes]), len(subclass_classes))
ex_train = train
# Second pass
temp = []
for tup in subclass_axioms:
    line, c, d = tup
    if train and (c in subclass_classes or d in subclass_classes):
#       print(line)
      file['train_norm'].write(line)
      file['train'].write(c + ' ' + d + '\n')
      count+=1
      train-=1
      if c in subclass_classes:
        subclass_classes.pop(c)
      if d in subclass_classes:
        subclass_classes.pop(d)
      if c not in classes:
        classes[c] = len(classes)
      if d not in classes:
        classes[d] = len(classes)
    else:
      temp.append((line, c, d))
subclass_axioms = temp
print(count+len(subclass_axioms)+len(other_axioms), len(subclass_axioms), len(other_axioms))
print("Added to train from subclass axioms", ex_train-train)
ex_train = train

print("remaining subclass axioms and other axioms", len(subclass_axioms), len(other_axioms))
print("train, valid, test", train, valid, test)

if len(subclass_axioms) < test+valid:
    # recalculate test, validation, training sample counts
    c = len(subclass_axioms)
    #valid = int(c*0.66)
    valid = int(c)
    #test = c - valid
   # if test>0:
    # valid+=test
     #test=0
    train = max(0, min(train, int(c*0.7/0.3) - (total_train-train)))
ex_train = train

print("Train, valid, test left", train, valid, test)

temp = []
while test and subclass_axioms != []:
    line, c, d = subclass_axioms.pop()
    if c not in subclass_classes and d not in subclass_classes:
      file['test'].write(c + ' ' + d + '\n')
      test-=1
      count+=1
    else:
      temp.append((line, c, d))
while valid and subclass_axioms != []:
    line, c, d =  subclass_axioms.pop()
    if c not in subclass_classes and d not in subclass_classes:
      file['valid'].write(c + ' ' + d + '\n')
      valid-=1
      count+=1
    else:
      temp.append((line, c, d))
subclass_axioms.extend(temp)
while train and other_axioms != []:
    line, r, c, d = other_axioms.pop()
#     if c in classes and d in classes:
    file['train'].write(c + ' ' + d + '\n')
    file['train_norm'].write(line)
    count+=1
    train-=1
print("added to train from other axioms", ex_train-train)
ex_train = train
while train and subclass_axioms != []:
    line, c, d =  subclass_axioms.pop()
#     if c in classes or d in classes:
    file['train'].write(c + ' ' + d + '\n')
    file['train_norm'].write(line)
    count+=1
    train -=1
print("added to train from subclass axioms", ex_train-train)
for k in ['train', 'valid', 'test']:
    file[k].close()
print("Total:", count)
print("left", len(subclass_axioms), len(other_axioms))
print(train, test, valid)

Running the above code gives two main file- train.txt, valid.txt, test.txt, which can be used in Meta-N-Ball Model.